In [1]:
import numpy as np
import random
import csv
import math 
import networkx as nx
import pandas as pd
from operator import itemgetter
from numpy import linalg as LA
import matplotlib.pyplot as plt
from random import uniform, seed
import time
from itertools import zip_longest
G=nx.Graph()
# f=open("fb-pages-sport.txt","r")
# edge1=[]
# edge2=[]
# nodelist=[]
# for x in f:
#     e1,e2=x.split()
#     edge1.append(int(e1)-1)
#     edge2.append(int(e2)-1)
#     for i in (int(e1),int(e2)):
#         if i not in nodelist:
#             nodelist.append(i)
f = open("bio-dmela.txt", "r")
edge1=[]
edge2=[]
nodelist=[]
for x in f:
  e1,e2=x.split()
  edge1.append(int(e1))
  edge2.append(int(e2))
  for i in (int(e1),int(e2)):
      if i not in nodelist:
          nodelist.append(i)
n=len(nodelist)

for  i in range(0, len(edge1)):
    G.add_edge(edge1[i], edge2[i])

import numpy as np
import random

# Define a function to simulate the SIR model for one time step
def sir_step(graph, state, p1, p2):
    # Initialize the set of nodes that are infected in this time step
    infected = set()
    
    # Initialize the set of nodes that have recovered in this time step
    recovered = set()
    
    # Loop through all nodes in the graph
    for node in graph:
        # If the node is susceptible
        if state[node] == 'S':
            # Loop through the node's neighbors
            for neighbor in graph[node]:
                # If the neighbor is infected
                if state[neighbor] == 'I':
                    # Generate a random number
                    rand = np.random.random()
                    # If the number is less than the infection probability
                    if rand < p1:
                        # Add the node to the set of infected nodes
                        infected.add(node)
                        # Break out of the loop through the neighbors
                        break
        
        # If the node is infected
        elif state[node] == 'I':
            # Generate a random number
            rand = np.random.random()
            # If the number is less than the recovery probability
            if rand < p2:
                # Add the node to the set of recovered nodes
                recovered.add(node)
    
    # Loop through the infected nodes and update their state to 'I'
    for node in infected:
        state[node] = 'I'
    
    # Loop through the recovered nodes and update their state to 'R'
    for node in recovered:
        state[node] = 'R'
    
    # Return the set of newly infected nodes
    return infected

# Define a function to simulate the SIR model for a specified number of time steps
def sir_simulation(graph, seeds, p1, p2, num_steps, num_sims):
    # Initialize the state of all nodes in the graph to 'S'
    state = {node: 'S' for node in graph}
    # Initialize the state of the seed nodes to 'I'
    for seed in seeds:
        state[seed] = 'I'
    # Initialize the list of infected sets for each time step
    infected_lists = [set(seeds)]
    countA=[]
    # Loop through the number of Monte Carlo simulations
    for i in range(num_sims):
        # Initialize the state of all nodes in the graph to 'S'
        state = {node: 'S' for node in graph}
        # Initialize the state of the seed nodes to 'I'
        for seed in seeds:
            state[seed] = 'I'
        
        # Initialize a list to store the number of infected nodes for each time step
        infected_counts = [len(seeds)]
        
        # Loop through the number of time steps
        for j in range(num_steps):
            # Run the SIR model for one time step
            newly_infected = sir_step(graph, state, p1, p2)
            # Add the newly infected nodes to the infected set
            infected_lists[-1].update(newly_infected)
            # Add the number of newly infected nodes to the infected_counts list
            infected_counts.append(len(infected_lists[-1]))
        countA.append([x for x in infected_counts])
        # Write the infected_counts list to a file
        #with open(filename, 'a') as f:
        #    f.write(','.join([str(x) for x in infected_counts]) + '\n')
        
        # Add an empty set to the infected_lists for the next simulation
        infected_lists.append(set())
    return infected_lists,countA

seednodes=[214, 662, 707, 75, 60, 1410, 239, 330, 208, 298]
          
beta=[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40]

totinf=[]
for i in beta:
    act,count=sir_simulation(G,seednodes,i,i,40,100)
    #print(count)
    a=np.mean(count, axis=0)
    totinf.append(a[-1])
    df = pd.DataFrame.from_dict({'time':[x for x in range(1, len(a)+1)],'cuminf':a})
    df.to_excel('avginf_SIR_BC_'+str(i)+'.xlsx', header=True, index=False)
    
df = pd.DataFrame.from_dict({'diffbeta':beta,'totinf':totinf})
df.to_excel('Difftotinf_SIR_BC.xlsx', header=True, index=False)
